In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import pattern
from pattern.text.en import singularize

**Part A I get 6587 features for yelp reviews.**

In [2]:
reviews:pd.DataFrame=pd.read_csv("mcdonalds-yelp-negative-reviews.csv",encoding='latin1')

**Regex Cleaning**

In [3]:
AH_PATTERN = re.compile(r'\bA{1,20}H{1,20}\b', flags=re.IGNORECASE)
uh_PATTERN = re.compile(r'\bU{1,20}H{1,20}\b', flags=re.IGNORECASE)
ew_PATTERN = re.compile(r'\bE{1,20}W{1,20}\w+\b', flags=re.IGNORECASE)
Time_pattern=re.compile(r'\b(\d+(?::\d+)?(?:/\d+)?(?::\d+)?[\s]?(?:A|P)M)\b', flags=re.IGNORECASE)
noise_number=re.compile(r'(\$?\d+(?:\w+)?)')
underscore=re.compile(r'(\b.?_\w+?)')
ugh_PATTERN = re.compile(r'\bU{1,20}G{1,20}h{0,20}\b', flags=re.IGNORECASE)
slow_PATTERN = re.compile(r'\bS{1,20}L{1,20}O{1,20}W{1,20}\b', flags=re.IGNORECASE)
too_PATTERN = re.compile(r'\bT{1,20}O{1,20}\b', flags=re.IGNORECASE)
so_PATTERN = re.compile(r'\bS{1,20}O{1,20}\b', flags=re.IGNORECASE)
yo_PATTERN = re.compile(r'\by(?:e|o)(?:a|p)[h]?\b', flags=re.IGNORECASE)
all_PATTERN = re.compile(r'\bA{1,20}L{1,20}\b', flags=re.IGNORECASE)
top_PATTERN = re.compile(r'\btop[s]?(?:ped|ping)?\b', flags=re.IGNORECASE)
still_PATTERN = re.compile(r'\bstil{1,20}\b', flags=re.IGNORECASE)
wait_PATTERN = re.compile(r'\bwait(?:ing|ed)?\b', flags=re.IGNORECASE)
spill_PATTERN = re.compile(r'\bspill(?:ing|ed)?\b', flags=re.IGNORECASE)
gr_PATTERN = re.compile(r'\bG{1,20}R{1,20}\b', flags=re.IGNORECASE)
spend_PATTERN = re.compile(r'\bspen(?:t|ding)?\b', flags=re.IGNORECASE)
why_PATTERN = re.compile(r'\bwhy{1,20}\b', flags=re.IGNORECASE)
way_PATTERN = re.compile(r'\bway{1,20}\b', flags=re.IGNORECASE)
woo_PATTERN = re.compile(r'\bwo{1,20}\b', flags=re.IGNORECASE)
grill_PATTERN = re.compile(r'\bgrill(?:ed|ing)?\b', flags=re.IGNORECASE)
worry_PATTERN = re.compile(r'\bworr(?:y|ied)?\b', flags=re.IGNORECASE)
work_PATTERN = re.compile(r'\bwork(?:ed|ing)?\b', flags=re.IGNORECASE)

In [4]:
reviews['review'] = reviews.review.str.replace(AH_PATTERN, '_ahhh_token_')
reviews['review'] = reviews.review.str.replace(uh_PATTERN, 'uh')
reviews['review'] = reviews.review.str.replace(Time_pattern, 'time')
reviews['review'] = reviews.review.str.replace(noise_number, 'number')
reviews['review'] = reviews.review.str.replace(underscore, 'underscore')
reviews['review'] = reviews.review.str.replace(ugh_PATTERN, 'ugh')
reviews['review'] = reviews.review.str.replace('wikipedium', 'wikipedia')
reviews['review'] = reviews.review.str.replace('yum', 'yummy')
reviews['review'] = reviews.review.str.replace(slow_PATTERN, 'slow')
reviews['review'] = reviews.review.str.replace(ew_PATTERN, 'ew')
reviews['review'] = reviews.review.str.replace(too_PATTERN, 'too')
reviews['review'] = reviews.review.str.replace(so_PATTERN, 'so')
reviews['review'] = reviews.review.str.replace(yo_PATTERN, 'y')
reviews['review'] = reviews.review.str.replace(all_PATTERN, 'all')
reviews['review'] = reviews.review.str.replace(top_PATTERN, 'top')
reviews['review'] = reviews.review.str.replace(still_PATTERN, 'still')
reviews['review'] = reviews.review.str.replace(wait_PATTERN, 'wait')
reviews['review'] = reviews.review.str.replace('thorugh', 'thorough')
reviews['review'] = reviews.review.str.replace('thrown', 'throw')
reviews['review'] = reviews.review.str.replace(spill_PATTERN, 'spill')
reviews['review'] = reviews.review.str.replace('tastele', 'tasteless')
reviews['review'] = reviews.review.str.replace('tasteles', 'tasteless')
reviews['review'] = reviews.review.str.replace(gr_PATTERN, 'gr')
reviews['review'] = reviews.review.str.replace('tastey', 'tasty')
reviews['review'] = reviews.review.str.replace('tasted', 'taste')
reviews['review'] = reviews.review.str.replace(spend_PATTERN, 'spend')
reviews['review'] = reviews.review.str.replace(why_PATTERN, 'why')
reviews['review'] = reviews.review.str.replace(way_PATTERN, 'way')
reviews['review'] = reviews.review.str.replace(woo_PATTERN, 'woo')
reviews['review'] = reviews.review.str.replace(grill_PATTERN, 'grill')
reviews['review'] = reviews.review.str.replace(worry_PATTERN, 'worry')
reviews['review'] = reviews.review.str.replace(work_PATTERN, 'work')
reviews['review'] = reviews.review.str.replace('saw', 'see')

In [5]:
reviews=reviews['review']

In [6]:
stopwords=stopwords.words('english')

**Add custom stopwords and Remove some stopwords for business purpose**

In [7]:
# ADD custom stopwords
Add_stopwords=['actually','usually','really','mcds','''mcdonald's''',
               'mcdonald','mcdonalds','''i'm''','''i've''','_ahhh_token_','time','number',
            'abbreviate','able','abour','underscore','î_','uh','ew','gr','woo']
for i in Add_stopwords:
    stopwords.append(i)

In [8]:
# Remove some stopwords for business purpose 
Not_remove=['no', 'nor', 'not','don', "don't",'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 
            'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't",
           'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't",
           'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
for i in Not_remove:
    stopwords.remove(i)

In [9]:
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', '

In [10]:
cleaned_reviews = []
for review in reviews:
    words = review.split()
    new_words = []
    for word in words:
        word=singularize(word).lower()
        if word in stopwords:
            continue
        new_words.append(word)
    cleaned_review = " ".join(new_words)
    cleaned_reviews.append(cleaned_review)

**Lemmatization with pos tag code is coming from gaurav5430's Github Gist**

**The reason** I used lemmatization is we are doing a review text analysis and I hope the result of this analysis can more precisely reflect the true sentiment of the reviews. Using stemming may increase false positive rate and reduce precision since stemming  is trying to chop off the ends of words and will count some words which have different meanings into same root. Lemmatization will return the base or dictionary form of a word, which means it can match words more precisely.

In [11]:
lemmatizer = WordNetLemmatizer()
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

In [12]:
def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

In [13]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/qiongqionglin/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/qiongqionglin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [14]:
df_lemma=[]
for i in cleaned_reviews:
    lemma_s=lemmatize_sentence(i)
    df_lemma.append(lemma_s)

In [15]:
# after lemmatizing, some words will change from non-stopword to stopwords -> abbreviated ->abbreviate
# so I remove stopwords again after lemmatizing
final_reviews = []
for review in df_lemma:
    words2 = review.split()
    new_words2 = []
    for word2 in words2:
        word2=singularize(word2)
        if word2 in stopwords:
            continue
        new_words2.append(word2)
    final_review = " ".join(new_words2)
    final_reviews.append(final_review)

In [16]:
vectorizer = CountVectorizer()
vectorizer.fit(final_reviews)

CountVectorizer()

In [17]:
X = vectorizer.transform(final_reviews)

In [18]:
print(X.shape)

(1525, 6677)


In [19]:
corpus_df= pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())

In [20]:
original_columns = corpus_df.columns 
to_drop_columns = set(original_columns).intersection(set(stopwords)) # get the list of words to drop
print(f"Dataframe shape was {corpus_df.shape}")
corpus_df.drop(columns=to_drop_columns, inplace=True)
print(f"Dataframe shape is now{corpus_df.shape}")

Dataframe shape was (1525, 6677)
Dataframe shape is now(1525, 6587)


In [22]:
corpus_df.describe()

,abc,ability,abmy,abode,abram,abrasive,absolute,absolutely,absorb,absurd,...,yuppie,zak,zax,zee,zeke,zero,zesty,zip,zombie,zoom
count,1525.000000,1525.000000,1525.000000,1525.000000,1525.000000,1525.000000,1525.000000,1525.000000,1525.000000,1525.000000,...,1525.000000,1525.000000,1525.000000,1525.000000,1525.000000,1525.000000,1525.000000,1525.000000,1525.000000,1525.000000
mean,0.000656,0.001967,0.000656,0.000656,0.000656,0.000656,0.006557,0.019016,0.000656,0.002623,...,0.000656,0.000656,0.000656,0.000656,0.000656,0.012459,0.000656,0.000656,0.004590,0.000656
std,0.025607,0.044324,0.025607,0.025607,0.025607,0.025607,0.088493,0.162914,0.025607,0.051164,...,0.025607,0.025607,0.025607,0.025607,0.025607,0.110959,0.025607,0.025607,0.092245,0.025607
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,3.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000,1.000000


**Part B**

In [23]:
with open('tale-of-two-cities.txt', 'r') as file:
    data = file.read().replace('\n', '')

In [24]:
sent_text = nltk.sent_tokenize(data)

**Perform stemming and then count-vectorization, I get 12895 features.**

In [25]:
stemmer = nltk.stem.porter.PorterStemmer()
df_stem =[]
for line in sent_text:
    words = word_tokenize(line)
    new_words = []
    for word in words:
        new_words.append(stemmer.stem(word))
    new_line =" ".join(new_words)
    df_stem.append(new_line)

In [26]:
vectorizer = CountVectorizer(token_pattern=r"(?u)\b\w\w+\b|!|\?|\"|\'")
vector1=vectorizer.fit_transform(df_stem)
df1= pd.DataFrame(vector1.toarray(), columns=vectorizer.get_feature_names())

In [27]:
df1.describe()

,!,',1757,1767,1792,21,?,aback,abandon,abandonedthat,...,youtouch,youuncomfort,youwant,youwear,youwel,youwhat,youwil,youwith,youwould,zealou
count,7543.000000,7543.000000,7543.000000,7543.000000,7543.000000,7543.000000,7543.000000,7543.000000,7543.000000,7543.000000,...,7543.000000,7543.000000,7543.000000,7543.000000,7543.000000,7543.000000,7543.000000,7543.000000,7543.000000,7543.000000
mean,0.126342,0.911308,0.000133,0.000265,0.000133,0.000133,0.120774,0.000133,0.001591,0.000133,...,0.000265,0.000133,0.000133,0.000133,0.000133,0.000133,0.000265,0.000133,0.000133,0.000265
std,0.336618,1.217615,0.011514,0.016282,0.011514,0.011514,0.328318,0.011514,0.039857,0.011514,...,0.016282,0.011514,0.011514,0.011514,0.011514,0.011514,0.016282,0.011514,0.011514,0.016282
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.000000,12.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


**Perform lemmatization and then count-vectorization, I get 14030 features.**

In [28]:
df_lemma2=[]
for line in sent_text:
    lemma=lemmatize_sentence(line)
    df_lemma2.append(lemma)

In [29]:
vent = CountVectorizer(token_pattern=r"(?u)\b\w\w+\b|!|\?|\"|\'")
vector2=vent.fit_transform(df_lemma2)
df2= pd.DataFrame(vector2.toarray(), columns=vent.get_feature_names())

In [30]:
df2.describe()

,!,',1757,1767,1792,21,?,aback,abandon,abandoned,...,youtouch,youuncomfortable,youwant,youwear,youwell,youwhat,youwill,youwith,youwould,zealous
count,7543.000000,7543.000000,7543.000000,7543.000000,7543.000000,7543.000000,7543.000000,7543.000000,7543.000000,7543.000000,...,7543.000000,7543.000000,7543.000000,7543.000000,7543.000000,7543.000000,7543.000000,7543.000000,7543.000000,7543.000000
mean,0.126342,0.911308,0.000133,0.000265,0.000133,0.000133,0.120774,0.000133,0.001326,0.000133,...,0.000265,0.000133,0.000133,0.000133,0.000133,0.000133,0.000265,0.000133,0.000133,0.000265
std,0.336618,1.217615,0.011514,0.016282,0.011514,0.011514,0.328318,0.011514,0.036389,0.011514,...,0.016282,0.011514,0.011514,0.011514,0.011514,0.011514,0.016282,0.011514,0.011514,0.016282
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.000000,12.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [31]:
# how to remove punctuation in df str.replace('[^\w\s]','')

**Perform lemmatization, remove stopwords, remove punctuation, and then perform count-vectorization, I get 13747 features.**

In [32]:
df_lemma3=[]
for line in sent_text:
    lemma3=lemmatize_sentence(line)
    df_lemma3.append(lemma3)

In [33]:
v = CountVectorizer(stop_words='english')
vector3=v.fit_transform(df_lemma3)
df3= pd.DataFrame(vector3.toarray(), columns=v.get_feature_names())

In [34]:
df3.describe()

,1757,1767,1792,21,aback,abandon,abandoned,abandonedthat,abandonment,abank,...,youtouch,youuncomfortable,youwant,youwear,youwell,youwhat,youwill,youwith,youwould,zealous
count,7543.000000,7543.000000,7543.000000,7543.000000,7543.000000,7543.000000,7543.000000,7543.000000,7543.000000,7543.000000,...,7543.000000,7543.000000,7543.000000,7543.000000,7543.000000,7543.000000,7543.000000,7543.000000,7543.000000,7543.000000
mean,0.000133,0.000265,0.000133,0.000133,0.000133,0.001326,0.000133,0.000133,0.000133,0.000133,...,0.000265,0.000133,0.000133,0.000133,0.000133,0.000133,0.000265,0.000133,0.000133,0.000265
std,0.011514,0.016282,0.011514,0.011514,0.011514,0.036389,0.011514,0.011514,0.011514,0.011514,...,0.016282,0.011514,0.011514,0.011514,0.011514,0.011514,0.016282,0.011514,0.011514,0.016282
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
